In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle


In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float32',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float32',
        'AVProductsEnabled':                                    'float32',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float32',
        'GeoNameIdentifier':                                    'float32',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float32',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float32',
        'IeVerIdentifier':                                      'float32',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float32',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float32',
        'Census_ProcessorManufacturerIdentifier':               'float32',
        'Census_ProcessorModelIdentifier':                      'float32',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float32',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float32',
        'Census_IsFlightsDisabled':                             'float32',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float32',
        'Census_FirmwareManufacturerIdentifier':                'float32',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float32',
        'Census_IsVirtualDevice':                               'float32',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float32',
        'Wdft_IsGamer':                                         'float32',
        'Wdft_RegionIdentifier':                                'float32',
        'HasDetections':                                        'int8'
        }

In [5]:
train = pd.read_csv(r'C:\Users\jithi\OneDrive\Desktop\ML project\microsoft-malware-prediction\train.csv',dtype=dtypes)

KeyboardInterrupt: 

In [4]:
# cols_to_drop = ['IsBeta','IsSxsPassiveMode','DefaultBrowsersIdentifier','Census_ProcessorClass','MachineIdentifier']
columns_to_drop = ['MachineIdentifier']
columns_to_split = ['EngineVersion','AppVersion','AvSigVersion','Census_OSVersion']
columns_for_OE = ['ProductName','RtpStateBitfield','Platform','Processor','OsVer','OsBuild','OsSuite','OsPlatformSubRelease','OsBuildLab','SkuEdition','PuaMode','SmartScreen','Census_MDC2FormFactor',
                  'Census_DeviceFamily','Census_ProcessorClass','Census_PrimaryDiskTypeName','Census_ChassisTypeName','Census_InternalPrimaryDiagonalDisplaySizeInInches','Census_InternalPrimaryDisplayResolutionHorizontal','Census_InternalPrimaryDisplayResolutionVertical',
                  'Census_PowerPlatformRoleName','Census_InternalBatteryType','Census_OSArchitecture','Census_OSBranch','Census_OSEdition','Census_OSSkuName','Census_OSInstallTypeName','Census_OSWUAutoUpdateOptionsName','Census_GenuineStateName','Census_ActivationChannel',
                   'Census_FlightRing',
                  ]
for col in columns_to_drop:
    train.drop(columns=[col], inplace=True)
cols_to_bin=['Census_PrimaryDiskTotalCapacity','Census_SystemVolumeTotalCapacity', 'Census_TotalPhysicalRAM']
for col in columns_to_split:
    split_cols = train[col].str.split('.', expand=True)  # Split column values by "."
    split_cols.columns = [f"{col}_{i+1}" for i in range(split_cols.shape[1])]  # Rename columns
    train = train.drop(columns=[col]).join(split_cols)  # Replace original column with new ones


oe = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
oe.fit(train[columns_for_OE])
train[columns_for_OE] = oe.transform(train[columns_for_OE])
from sklearn.preprocessing import OrdinalEncoder

with open('ordinal_encoder.pkl', 'wb') as f:
    pickle.dump(oe, f)

train.drop(index=5244810, inplace=True)

for col in train.columns:
    if train[col].dtype == 'float64':
        train[col] = train[col].astype('float32')
    elif train[col].dtype == 'object':
        train[col] = train[col].astype('float32')



y = train['HasDetections']
X = train.drop(columns=['HasDetections'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#save scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
    

In [7]:
#train data and labels combined

train_data = pd.DataFrame(X_train, columns=X.columns)
train_data['HasDetections'] = y_train.values

test_data = pd.DataFrame(X_test, columns=X.columns)
test_data['HasDetections'] = y_test.values



In [11]:
import pandas as pd
import numpy as np

def optimize_dataframe(df):
    for col in df.columns:
        col_data = df[col]
        
        # Skip non-numeric columns
        if not np.issubdtype(col_data.dtype, np.number):
            continue

        # Check if all values are integers (i.e., no decimal part)
        if np.all(col_data.dropna() == col_data.dropna().astype(int)):
            df[col] = pd.to_numeric(col_data, downcast="integer")
        else:
            df[col] = pd.to_numeric(col_data, downcast="float")
    
    return df

# Apply optimization
train_data = optimize_dataframe(train_data)
test_data = optimize_dataframe(test_data)

# Check memory usage before and after
print(f"Memory usage after optimization: {train_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Memory usage after optimization: {test_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Memory usage after optimization: 2457.16 MB
Memory usage after optimization: 614.29 MB


In [16]:
#save train and test now

train_data.to_csv('train_data_scaled_encoded_shrunk.csv', index=False)
test_data.to_csv('test_data_scaled_encoded_shrunk.csv', index=False)